Usar o XGBoost é fácil. Talvez muito fácil, considerando que geralmente é considerado o melhor algoritmo de ML do momento.

Para instalá-lo, basta:

pip instalar xgboost

Vamos experimentar usando o conjunto de dados Iris. Este conjunto de dados inclui a largura e o comprimento das pétalas e sépalas de muitas flores de íris e a espécie específica de íris à qual a flor pertence. Nosso desafio é prever as espécies de uma amostra de flor apenas com base no tamanho de suas pétalas. Revisitaremos esse conjunto de dados mais tarde, quando falarmos também sobre análise de componentes principais.

In [1]:
from sklearn.datasets import load_iris

# Carrega o conjunto de dados Iris da biblioteca scikit-learn
iris = load_iris()

# Obtém o número de amostras (linhas) e o número de características (colunas) dos dados
numSamples, numFeatures = iris.data.shape

# Exibe o número de amostras no conjunto de dados
print(numSamples)  # Esperado: 150

# Exibe o número de características por amostra
print(numFeatures)  # Esperado: 4

# Exibe a lista de nomes das classes (tipos de flores de íris)
print(list(iris.target_names))  # Esperado: ['setosa', 'versicolor', 'virginica']


150
4
['setosa', 'versicolor', 'virginica']


Vamos dividir nossos dados em 20% reservados para testar nosso modelo e os 80% restantes para treiná-lo. Ao reter nossos dados de teste, podemos ter certeza de que estamos avaliando seus resultados com base em novas flores nunca vistas antes. Normalmente nos referimos às nossas características (neste caso, os tamanhos das pétalas) como X, e aos rótulos (neste caso, as espécies) como y.

In [2]:
from sklearn.model_selection import train_test_split

# Divide o conjunto de dados iris em conjuntos de treino e teste
# 80% dos dados serão usados para treinamento (X_train, y_train) e 20% para teste (X_test, y_test)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)


Agora carregaremos o XGBoost e converteremos nossos dados no formato DMatrix esperado. Um para os dados de treinamento e outro para os dados de teste.

In [3]:
import xgboost as xgb

# Cria a matriz DMatrix para os dados de treino
train = xgb.DMatrix(X_train, label=y_train)

# Cria a matriz DMatrix para os dados de teste
test = xgb.DMatrix(X_test, label=y_test)


Agora definiremos nossos hiperparâmetros. Estamos escolhendo softmax porque este é um problema de classificação múltipla, mas o ideal é que os outros parâmetros sejam ajustados por meio de experimentação.

In [4]:
# Definindo os parâmetros para o modelo XGBoost
param = {
    'max_depth': 4,  # Profundidade máxima das árvores de decisão. Um valor mais alto pode aumentar a capacidade do modelo de se ajustar aos dados, mas também aumenta o risco de overfitting.
    'eta': 0.3,  # Taxa de aprendizado. Controla o peso das novas árvores adicionadas ao modelo. Valores menores resultam em aprendizado mais lento e preciso, mas podem exigir mais épocas.
    'objective': 'multi:softmax',  # Função objetivo usada para classificação multiclasse. O 'softmax' retorna o índice da classe com a maior probabilidade.
    'num_class': 3  # Número de classes no problema de classificação (no caso do conjunto de dados Iris, temos 3 classes de flores).
}

# Número de épocas (iterações de treinamento)
epochs = 10  # Número de passagens pelos dados de treino. Mais épocas podem melh


Vamos treinar nosso modelo usando esses parâmetros como uma primeira estimativa.

In [5]:
# Treinando o modelo XGBoost
model = xgb.train(param, train, epochs)


Agora usaremos o modelo treinado para prever classificações dos dados que reservamos para teste. Cada número de classificação que recebemos corresponde a uma espécie específica de íris.

In [6]:
predictions = model.predict(test)


In [7]:
print(predictions)

[2. 1. 0. 2. 0. 2. 0. 1. 1. 1. 2. 1. 1. 1. 1. 0. 1. 1. 0. 0. 2. 1. 0. 0.
 2. 0. 0. 1. 1. 0.]


Vamos medir a precisão dos dados de teste...

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)

1.0

Santo corvo! É perfeito, e isso só nos permite adivinhar os melhores hiperparâmetros!

Normalmente eu faria você experimentar para encontrar hiperparâmetros melhores como atividade, mas você não pode melhorar esses resultados. Em vez disso, veja o que é necessário para piorar os resultados! Com quantas épocas (iterações) posso me safar? Quão baixo posso definir o max_profundidade? Basicamente tente otimizar a simplicidade e o desempenho do modelo, agora que você já tem uma precisão perfeita

In [9]:
import numpy as np

# Testando diferentes valores de max_depth e epochs
max_depth_values = [1, 2, 3, 4]  # profundidades a serem testadas
epoch_values = [1, 2, 5, 10]  # épocas a serem testadas

results = {}

# Loop sobre as profundidades
for max_depth in max_depth_values:
    # Loop sobre as épocas
    for epochs in epoch_values:
        param = {
            'max_depth': max_depth,
            'eta': 0.3,
            'objective': 'multi:softmax',
            'num_class': 3
        }
        
        # Treinando o modelo
        model = xgb.train(param, train, epochs)
        
        # Fazendo previsões
        predictions = model.predict(test)
        
        # Avaliando a precisão
        accuracy = np.sum(predictions == y_test) / len(y_test)
        
        # Armazenando os resultados
        results[(max_depth, epochs)] = accuracy

# Exibindo os resultados
for key, value in results.items():
    print(f"Max Depth: {key[0]}, Epochs: {key[1]}, Accuracy: {value:.2f}")


Max Depth: 1, Epochs: 1, Accuracy: 0.97
Max Depth: 1, Epochs: 2, Accuracy: 0.97
Max Depth: 1, Epochs: 5, Accuracy: 0.97
Max Depth: 1, Epochs: 10, Accuracy: 0.97
Max Depth: 2, Epochs: 1, Accuracy: 0.97
Max Depth: 2, Epochs: 2, Accuracy: 1.00
Max Depth: 2, Epochs: 5, Accuracy: 0.97
Max Depth: 2, Epochs: 10, Accuracy: 0.97
Max Depth: 3, Epochs: 1, Accuracy: 1.00
Max Depth: 3, Epochs: 2, Accuracy: 1.00
Max Depth: 3, Epochs: 5, Accuracy: 1.00
Max Depth: 3, Epochs: 10, Accuracy: 0.97
Max Depth: 4, Epochs: 1, Accuracy: 1.00
Max Depth: 4, Epochs: 2, Accuracy: 1.00
Max Depth: 4, Epochs: 5, Accuracy: 1.00
Max Depth: 4, Epochs: 10, Accuracy: 1.00
